In [13]:
import pandas as pd
import numpy as np
import tqdm

In [ ]:
ratings = pd.read_csv('rating.csv', parse_dates=['timestamp'])

ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
rand_userIds = np.random.choice(ratings['userId'].unique(), size=int(len(ratings['userId'].unique())*0.3), replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]

print(len(ratings), len(rand_userIds))

6004726 41547


In [7]:
ratings.sample(5)

,userId,movieId,rating,timestamp
18874132,130694,1196,4.5,2005-01-16 21:38:27
9609484,66486,1721,3.0,2001-09-13 02:47:53
1919586,12983,2605,4.0,2000-08-04 00:30:33
18503660,128082,356,2.0,2005-03-23 01:11:07
12173394,84086,3147,4.0,2006-09-10 22:21:01


--> Splitting the dataset

In [8]:
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

In [9]:
train_ratings

,userId,movieId,rating
517,6,1,5.0
518,6,3,3.0
519,6,7,5.0
520,6,17,5.0
521,6,52,5.0
...,...,...,...
19999802,138491,3608,3.5
19999803,138491,4128,4.0
19999804,138491,6874,4.0
19999806,138491,33794,2.5


In [10]:
test_ratings

,userId,movieId,rating
524,6,140,4.0
703,7,2762,5.0
896,9,2688,3.0
2072,19,67,3.0
2315,22,303,3.0
...,...,...,...
19999314,138485,48516,5.0
19999482,138486,8957,5.0
19999554,138487,2701,3.0
19999578,138488,207,3.0


In [11]:
train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

,userId,movieId,rating
14877704,102793,2082,1.0
10635351,73590,1645,1.0
11089886,76630,5693,1.0
9525153,65919,1674,1.0
19629396,135889,47,1.0


In [14]:
# Get a list of all movie IDs
all_movieIds = ratings['movieId'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

TypeError: 'module' object is not callable